## Import

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TreeCodeAlong").getOrCreate()

## Load data

It is about predicting if university is public or private. Input: university features, Output: "public" or "private"

In [2]:
ls

 College.csv*             'Tree Methods Code Along.ipynb'*
 dog_food.csv*             Tree_Methods_Consulting_Project.ipynb*
 My_code_along.ipynb*      Tree_Methods_Consulting_Project_SOLUTION.ipynb*
 My_doc_example.ipynb*     Tree_Methods_Doc_Example.ipynb*
 sample_libsvm_data.txt*


In [3]:
df = spark.read.csv("College.csv", inferSchema=True, header=True)

In [4]:
df.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|
|      Adrian College|    Yes|1428|  1097|   336|       22|       50|       1036|         99|  

In [5]:
print(df.count(), len(df.columns))

777 19


In [6]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



 - Good: all important features (we dont't need the school name) are numeric
 - In pyspark ml, all features need to be combine into one "column" using vectorAnsembler

## Data preparation

### Step 1: Prepare features

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [12]:
# create a vector assembler object
VecAss = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol="features")

In [16]:
# what the code below does: take df as input, create a new column called "features" using input from input columns
output1 = VecAss.transform(df)

In [18]:
output1.show(5)

+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
|              School|Private|Apps|Accept|Enroll|Top10perc|Top25perc|F_Undergrad|P_Undergrad|Outstate|Room_Board|Books|Personal|PhD|Terminal|S_F_Ratio|perc_alumni|Expend|Grad_Rate|            features|
+--------------------+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+
|Abilene Christian...|    Yes|1660|  1232|   721|       23|       52|       2885|        537|    7440|      3300|  450|    2200| 70|      78|     18.1|         12|  7041|       60|[1660.0,1232.0,72...|
|  Adelphi University|    Yes|2186|  1924|   512|       16|       29|       2683|       1227|   12280|      6450|  750|    1500| 29|      30|     12.2|         16| 10527|       56|[2186.0,1924

### Step 2: Prepare label

In [19]:
from pyspark.ml.feature import StringIndexer

In [20]:
StrIndexer = StringIndexer(inputCol="Private", outputCol="PrivateIndexer")

In [21]:
output2 = StrIndexer.fit(output1).transform(output1)

In [22]:
# check the data schema, important are columns "features" and "PrivateIndexer" at the end
output2.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndexer: double (nullable = false)



In [23]:
data = output2.select(["PrivateIndexer", "features"])

In [24]:
train_data, test_data = data.randomSplit([0.7, 0.3])

## Training

In [25]:
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier

In [26]:
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="PrivateIndexer")
rfc = RandomForestClassifier(featuresCol="features", labelCol="PrivateIndexer")
gbc = GBTClassifier(featuresCol="features", labelCol="PrivateIndexer")

In [28]:
tree_model = dtc.fit(train_data)

In [29]:
forest_model = rfc.fit(train_data)

In [30]:
gb_model = gbc.fit(train_data)

## Prediction

In [31]:
tree_test_pred = tree_model.transform(test_data)
forest_test_pred = forest_model.transform(test_data)
gb_test_pred = gb_model.transform(test_data)

In [41]:
tree_test_pred.show(5)

+--------------+--------------------+-------------+--------------------+----------+
|PrivateIndexer|            features|rawPrediction|         probability|prediction|
+--------------+--------------------+-------------+--------------------+----------+
|           0.0|[81.0,72.0,51.0,3...|  [248.0,0.0]|           [1.0,0.0]|       0.0|
|           0.0|[174.0,146.0,88.0...|  [248.0,0.0]|           [1.0,0.0]|       0.0|
|           0.0|[232.0,182.0,99.0...|   [33.0,2.0]|[0.94285714285714...|       0.0|
|           0.0|[235.0,217.0,121....|  [248.0,0.0]|           [1.0,0.0]|       0.0|
|           0.0|[257.0,183.0,109....|   [33.0,2.0]|[0.94285714285714...|       0.0|
+--------------+--------------------+-------------+--------------------+----------+
only showing top 5 rows



In [42]:
forest_test_pred.show(5)

+--------------+--------------------+--------------------+--------------------+----------+
|PrivateIndexer|            features|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+--------------------+----------+
|           0.0|[81.0,72.0,51.0,3...|[16.8775383428685...|[0.84387691714342...|       0.0|
|           0.0|[174.0,146.0,88.0...|[18.8061008190663...|[0.94030504095331...|       0.0|
|           0.0|[232.0,182.0,99.0...|[17.0967304789834...|[0.85483652394917...|       0.0|
|           0.0|[235.0,217.0,121....|[19.8635896412086...|[0.99317948206043...|       0.0|
|           0.0|[257.0,183.0,109....|[19.2762552457958...|[0.96381276228979...|       0.0|
+--------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [43]:
gb_test_pred.show(5)

+--------------+--------------------+--------------------+--------------------+----------+
|PrivateIndexer|            features|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+--------------------+----------+
|           0.0|[81.0,72.0,51.0,3...|[0.45354666612650...|[0.71240500056787...|       0.0|
|           0.0|[174.0,146.0,88.0...|[0.13936504859285...|[0.56923486288911...|       0.0|
|           0.0|[232.0,182.0,99.0...|[1.19862641006967...|[0.91661757728289...|       0.0|
|           0.0|[235.0,217.0,121....|[1.56751427840079...|[0.95831473533668...|       0.0|
|           0.0|[257.0,183.0,109....|[1.41411041438339...|[0.94418190948936...|       0.0|
+--------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



## Evaluation

In [32]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [46]:
# note that the BinaryClassificationEvaluator doesn't allow metric "accuracy" and "f1"
my_eval = BinaryClassificationEvaluator(labelCol="PrivateIndexer", metricName="areaUnderROC")

In [47]:
print(my_eval.evaluate(tree_test_pred))

0.9526301419730616


In [48]:
print(my_eval.evaluate(forest_test_pred))

0.9810702584637787


In [49]:
print(my_eval.evaluate(gb_test_pred))

0.9700127411721878
